## Importing pandas, sklearn libraries and udfs

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
def title_apply(df, col, titles):
    for i in titles:
        if i in df[col]:
            return i

### Reading the data

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")

# Working with train data

### Transforming categorical to numeric

In [4]:
df_train['Sex'] = pd.Categorical(df_train['Sex'])
df_train['Sex_n'] = df_train['Sex'].cat.codes

df_train['Embarked'] = pd.Categorical(df_train['Embarked'])
df_train['Embarked_n'] = df_train['Embarked'].cat.codes

In [5]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_n,Embarked_n
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,1.529742
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990,0.800254
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000,1.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000,2.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,2.000000


### Fill age with mean value and Fare

In [6]:
# Age
df_train['Age'] = df_train['Age'].round(0)
df_train['Age'] = df_train['Age'].fillna(round(df_train['Age'].mean(),0))

# Fare
df_train['Fare'] = df_train['Fare'].fillna(0)

In [7]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_n,Embarked_n
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.754209,0.523008,0.381594,32.204208,0.647587,1.529742
std,257.353842,0.486592,0.836071,13.000828,1.102743,0.806057,49.693429,0.477990,0.800254
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,1.000000
50%,446.000000,0.000000,3.000000,30.000000,0.000000,0.000000,14.454200,1.000000,2.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,2.000000


### Generating new features
- Title

In [8]:
# Defining titles list
titles = []

for name in df_train['Name']:
    splitted = name.split(' ')
    for i in splitted:
        if '.' in i:
            titles.append(i)
            
titles = list(set(titles))
titles.remove('L.')
titles

['Jonkheer.',
 'Mrs.',
 'Mr.',
 'Ms.',
 'Mlle.',
 'Capt.',
 'Major.',
 'Sir.',
 'Miss.',
 'Countess.',
 'Dr.',
 'Rev.',
 'Master.',
 'Don.',
 'Mme.',
 'Lady.',
 'Col.']

In [9]:
# Applying new title column
df_train['Title'] = df_train.apply(lambda x: title_apply(x, 'Name', titles), axis=1)

# Categorical to numerical
df_train['Title'] = pd.Categorical(df_train['Title'])
df_train['Title_n'] = df_train['Title'].cat.codes

In [10]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_n,Embarked_n,Title,Title_n
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,2,Mr.,12
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,Mrs.,13
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,2,Miss.,9
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,2,Mrs.,13
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,2,Mr.,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,2,Rev.,15
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,2,Miss.,9
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,30.0,1,2,W./C. 6607,23.4500,NaN,S,0,2,Miss.,9
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,0,Mr.,12


### Feature selection and model training

In [11]:
features = ['Pclass','Sex_n','Age','SibSp','Parch','Fare','Embarked_n','Title_n']

train_X = df_train[features]
train_y = df_train['Survived']

In [12]:
model = DecisionTreeClassifier(random_state=28)
model.fit(train_X, train_y)

DecisionTreeClassifier(random_state=28)

In [13]:
train_predictions = pd.Series(model.predict(train_X))
train_predictions = train_predictions.rename('Predicted')

In [14]:
predicted = pd.concat([df_train['Survived'], train_predictions], axis=1)
predicted['Result'] = predicted.apply(lambda x: x['Survived'] == x['Predicted'], axis=1)

correct_predictions = predicted['Result'].value_counts()[True]
total = predicted['Result'].count()

model_accuracy = correct_predictions / total
print(f'Model accuracy is : {model_accuracy}')

Model accuracy is : 0.9809203142536476


### Applying the changes on test data

In [15]:
# Sex
df_test['Sex'] = pd.Categorical(df_test['Sex'])
df_test['Sex_n'] = df_test['Sex'].cat.codes

# Embarked
df_test['Embarked'] = pd.Categorical(df_test['Embarked'])
df_test['Embarked_n'] = df_test['Embarked'].cat.codes

# Age
df_test['Age'] = df_test['Age'].round(0)
df_test['Age'] = df_test['Age'].fillna(round(df_test['Age'].mean(),0))

# Fare
df_test['Fare'] = df_test['Fare'].fillna(0)

# Defining titles list
titles = []

for name in df_test['Name']:
    splitted = name.split(' ')
    for i in splitted:
        if '.' in i:
            titles.append(i)
            
titles = list(set(titles))

# Applying new title column
df_test['Title'] = df_test.apply(lambda x: title_apply(x, 'Name', titles), axis=1)

# Categorical to numerical
df_test['Title'] = pd.Categorical(df_test['Title'])
df_test['Title_n'] = df_test['Title'].cat.codes

In [16]:
# Test Feature selection
test_X = df_test[features]

# Predictions
test_predictions = model.predict(test_X)

In [17]:
df_submission = pd.concat([df_test['PassengerId'], pd.Series(test_predictions)], axis=1)

In [18]:
df_submission = df_submission.rename(columns={0:'Survived'})
df_submission.to_csv("titanic_results.csv", header=True, index=False)